<a href="https://colab.research.google.com/github/Saoudyahya/RHLF/blob/main/RHLF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

# Create a DataFrame with example prompts and feedback scores
data = {
    'prompt': [
        "What is the capital of France?",
        "Explain the theory of relativity in simple terms.",
        "Write a poem about the sea.",
        "Describe the process of photosynthesis.",
        "How does blockchain technology work?",
        "Give a summary of Shakespeare's 'Hamlet'.",
        "What are the health benefits of meditation?",
        "List the main causes of climate change.",
        "Describe the main characters in 'Pride and Prejudice'.",
        "Explain quantum computing in layman's terms."
    ],
    'feedback': [0.9, 0.8, 0.7, 0.95, 0.85, 0.88, 0.92, 0.8, 0.9, 0.75]
}

df = pd.DataFrame(data)

# Save to CSV
df.to_csv('human_feedback_dataset.csv', index=False)

print("Dataset created: human_feedback_dataset.csv")


Dataset created: human_feedback_dataset.csv


In [3]:
pip install  datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.1 MB/s eta 0:00:00


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset

# Load your dataset
dataset = pd.read_csv('human_feedback_dataset.csv')

# Initialize tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
reward_model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=1)

# Prepare the data
inputs = tokenizer(dataset['prompt'].tolist(), return_tensors='pt', padding=True, truncation=True)
labels = torch.tensor(dataset['feedback'].tolist()).unsqueeze(1)

# Train the reward model
optimizer = optim.Adam(reward_model.parameters(), lr=5e-5)
reward_model.train()

for epoch in range(3):  # Number of epochs
    optimizer.zero_grad()
    outputs = reward_model(**inputs).logits
    loss = nn.MSELoss()(outputs, labels.float())
    loss.backward()
    optimizer.step()
    print(f'Epoch {epoch + 1}: Loss = {loss.item()}')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1: Loss = 0.8306739926338196
Epoch 2: Loss = 0.46003109216690063
Epoch 3: Loss = 0.14102405309677124


In [9]:
import random
from transformers import AutoModelForCausalLM

# Load a causal language model
lm_model = AutoModelForCausalLM.from_pretrained('gpt2')

def generate_response(prompt):
    # Tokenize the input
    input_ids = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True)

    # Get the attention mask
    attention_mask = input_ids['attention_mask']

    # Generate response with specified padding token id
    outputs = lm_model.generate(
        input_ids['input_ids'],
        attention_mask=attention_mask,
        max_length=50,  # Adjust as needed
        pad_token_id=tokenizer.eos_token_id  # Set padding token id
    )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Clean up the response to remove any unwanted tokens
    cleaned_response = ' '.join(token for token in response.split() if not token.startswith('[unused'))
    return cleaned_response


In [10]:
import torch
import random

# Reinforcement learning loop
for episode in range(10):  # Number of episodes
    prompt = "What is the capital of France?"
    response = generate_response(prompt)

    # Simulate human feedback for demonstration purposes
    human_feedback = torch.tensor([random.uniform(0, 1)]).unsqueeze(0)

    print(f"Prompt: {prompt}, Response: {response}, Feedback: {human_feedback.item()}")


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Prompt: What is the capital of France?, Response: what is the capital of france? — cost france 示 ∗ victim everyone 金os 訁 confederate everyone 示, Feedback: 0.09039730578660965


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Prompt: What is the capital of France?, Response: what is the capital of france? — cost france 示 ∗ victim everyone 金os 訁 confederate everyone 示, Feedback: 0.0036446924787014723


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Prompt: What is the capital of France?, Response: what is the capital of france? — cost france 示 ∗ victim everyone 金os 訁 confederate everyone 示, Feedback: 0.881655216217041


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Prompt: What is the capital of France?, Response: what is the capital of france? — cost france 示 ∗ victim everyone 金os 訁 confederate everyone 示, Feedback: 0.6767144203186035


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Prompt: What is the capital of France?, Response: what is the capital of france? — cost france 示 ∗ victim everyone 金os 訁 confederate everyone 示, Feedback: 0.44848188757896423
Prompt: What is the capital of France?, Response: what is the capital of france? — cost france 示 ∗ victim everyone 金os 訁 confederate everyone 示, Feedback: 0.9945372939109802
